In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Ponto de coleta dos dados históricos: PINH04900

Ponto de coleta do último dado: Ponte Cidade Jardim

In [151]:
df_2012_2017 = pd.read_excel('PINH04900_2012_2017.xlsx', decimal=",")
df_2017_2022 = pd.read_excel('PINH04900_2017_2022.xlsx', decimal=",")

In [152]:
df_2012_2022 = pd.concat([df_2012_2017, df_2017_2022])
df_2012_2022.head()

,Período DE,Período ATE,Cod_Interaguas,Tipo Rede,UGRHI,Código Ponto,Status Ponto,Data Coleta,Hora Coleta,Parametro,...,CLASSE,Município,UF,Inicio Operação,Fim Operação,Latitude,Longitude,Altitude,Localização,Captação
0,08/11/2012,08/11/2017,122,Rede Básica,06 - ALTO TIÊTE,PINH04900,Ativo,12/11/2012,15:40,Chuvas nas últimas 24h,...,Classe 4,SÃO PAULO,SP,01/01/1998,NaN,23 31 52,46 44 54,720,"Próximo à sua foz no Rio Tietê, na Estrutura d...",N
1,08/11/2012,08/11/2017,122,Rede Básica,06 - ALTO TIÊTE,PINH04900,Ativo,21/01/2013,12:49,Chuvas nas últimas 24h,...,Classe 4,SÃO PAULO,SP,01/01/1998,NaN,23 31 52,46 44 54,720,"Próximo à sua foz no Rio Tietê, na Estrutura d...",N
2,08/11/2012,08/11/2017,122,Rede Básica,06 - ALTO TIÊTE,PINH04900,Ativo,18/03/2013,12:45,Chuvas nas últimas 24h,...,Classe 4,SÃO PAULO,SP,01/01/1998,NaN,23 31 52,46 44 54,720,"Próximo à sua foz no Rio Tietê, na Estrutura d...",N
3,08/11/2012,08/11/2017,122,Rede Básica,06 - ALTO TIÊTE,PINH04900,Ativo,06/05/2013,14:40,Chuvas nas últimas 24h,...,Classe 4,SÃO PAULO,SP,01/01/1998,NaN,23 31 52,46 44 54,720,"Próximo à sua foz no Rio Tietê, na Estrutura d...",N
4,08/11/2012,08/11/2017,122,Rede Básica,06 - ALTO TIÊTE,PINH04900,Ativo,15/07/2013,09:52,Chuvas nas últimas 24h,...,Classe 4,SÃO PAULO,SP,01/01/1998,NaN,23 31 52,46 44 54,720,"Próximo à sua foz no Rio Tietê, na Estrutura d...",N


In [153]:
df_2012_2022['datetime'] = pd.to_datetime(df_2012_2022['Data Coleta'] + ' ' + df_2012_2022['Hora Coleta'], format="%d/%m/%Y %H:%M")
df_2012_2022['datetime']

0      2012-11-12 15:40:00
1      2013-01-21 12:49:00
2      2013-03-18 12:45:00
3      2013-05-06 14:40:00
4      2013-07-15 09:52:00
               ...        
1651   2022-10-06 10:05:00
1652   2022-10-06 10:05:00
1653   2022-10-06 10:05:00
1654   2022-10-06 10:05:00
1655   2022-10-06 10:05:00
Name: datetime, Length: 3007, dtype: datetime64[ns]

In [154]:
df_2012_2022 = df_2012_2022.sort_values(by='datetime')

In [155]:
df_2012_2022_ph = df_2012_2022.loc[df_2012_2022['Parametro'] == 'pH']
df_2012_2022_coliformes = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Escherichia coli**']
df_2012_2022_dbo = df_2012_2022.loc[df_2012_2022['Parametro'] == 'DBO (5, 20)']
df_2012_2022_fosforo = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Fósforo Total']
df_2012_2022_nitrogenio = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Nitrogênio Kjeldahl']
df_2012_2022_oxigenio = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Oxigênio Dissolvido']
df_2012_2022_solidos = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Sólido Total']
df_2012_2022_temperatura_agua = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Temperatura da Água']
df_2012_2022_temperatura_ar = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Temperatura do Ar']
df_2012_2022_turbidez = df_2012_2022.loc[df_2012_2022['Parametro'] == 'Turbidez']

In [156]:
#Fator de correção para coliformes em água

df_2012_2022_coliformes['Valor'] = df_2012_2022_coliformes['Valor'].astype(float)
df_2012_2022_coliformes['Valor'] = df_2012_2022_coliformes['Valor'].div(80)

<ipython-input-156-bd174adc2f8c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2012_2022_coliformes['Valor'] = df_2012_2022_coliformes['Valor'].astype(float)
<ipython-input-156-bd174adc2f8c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2012_2022_coliformes['Valor'] = df_2012_2022_coliformes['Valor'].div(80)


In [157]:
variacao_temperatura = []
for n in range(0, len(df_2012_2022_temperatura_agua)):
    variacao_temperatura.append(float(df_2012_2022_temperatura_agua.iloc[n]['Valor']) - float(df_2012_2022_temperatura_ar.iloc[n]['Valor']))

In [158]:
df_2012_2022_iqa = pd.DataFrame(columns=['datetime','Valor'])
dfs = [df_2012_2022_ph, df_2012_2022_oxigenio, df_2012_2022_coliformes, df_2012_2022_dbo, df_2012_2022_nitrogenio, df_2012_2022_fosforo, df_2012_2022_temperatura_agua, df_2012_2022_solidos, df_2012_2022_turbidez]
iqa = [] 
dates = []
ws = [0.12, 0.17, 0.15, 0.10, 0.10, 0.10, 0.10, 0.08, 0.08]
for date in df_2012_2022["datetime"]:
    count = 0
    vals = []
    for i in range (len(dfs)):
        for j in range(len(dfs[i].datetime)):
            if dfs[i].datetime.values[j] == date:
                if i == 6:
                    vals.append(abs(variacao_temperatura[j]))
                else:
                    vals.append(float(dfs[i]["Valor"].values[j])**ws[i])
                count += 1
    if count == 9:
        dates.append(date)
        iqa.append(np.prod(vals))
    
df_2012_2022_iqa["datetime"] = dates
df_2012_2022_iqa["Valor"] = iqa
df_2012_2022_iqa


,datetime,Valor
0,2012-11-12 15:40:00,29.696511
1,2012-11-12 15:40:00,29.696511
2,2012-11-12 15:40:00,29.696511
3,2012-11-12 15:40:00,29.696511
4,2012-11-12 15:40:00,29.696511
...,...,...
1499,2022-07-20 08:50:00,13.903448
1500,2022-07-20 08:50:00,13.903448
1501,2022-07-20 08:50:00,13.903448
1502,2022-07-20 08:50:00,13.903448


In [159]:
df_2012_2022_iqa.to_csv("iqa.csv")